In [1]:
import netCDF4 as nc
import numpy as np
import time
import os
import xarray as xr

import matplotlib.pyplot as plt
import shapefile
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
import matplotlib.tri as tri


In [2]:
def CreateSurfaceDataset(ncid_inp,nc_out_name,grids_in_poly):
    # nc_out_name = infile.split('.')[0]+'_withinWS.nc'
    print('surface dataset:  ', nc_out_name)

    # ncid_inp = surfdata_sparse
    ncid_out = nc.Dataset(nc_out_name, 'w', format='NETCDF3_64BIT_OFFSET')   

    ndims=len(ncid_inp.dimensions)
    nvars=len(ncid_inp.variables)

    '''Define dimensions'''
    lonlat_found = 0
    dimname_list=[0]*ndims
    dim_list=[0]*ndims

    # print(dimname_list)
    for (idim,n) in zip(ncid_inp.dimensions,range(ndims)):
        # print(idim,n)
        dimname = ncid_inp.dimensions[idim].name
        dimname_list[n] = dimname
        dimlen = len(ncid_inp.dimensions[idim])
        if dimname in ['lsmlon','lsmlat']:
            if lonlat_found ==0:
                lonlat_found = 1
                dimname = 'gridcell'
                dimname_list.append(dimname)
                dimlen = len(lon_region)
                # print(dimname,dimlen)
                dim_list[n] = ncid_out.createDimension(dimname, dimlen)
        elif dimname == 'time':
            # print(dimname,dimlen)
            dimname_list.append(dimname)
            dim_list[n] =  ncid_out.createDimension(dimname,None);
        elif dimname == 'gridcell':
            dimlen = len(lon_elm_in)
            dimname_list.append(dimname)
            # print('elif',dimname,dimlen)
            dim_list[n] = ncid_out.createDimension(dimname, dimlen)
        else:
            # print('else',dimname,dimlen)
            dim_list[n] = ncid_out.createDimension(dimname, dimlen)

    '''Define variables'''
    varnames = []
    varids = [0]*nvars
    for (varid,ivar) in zip(range(nvars),ncid_inp.variables):
        varname = ncid_inp.variables[ivar].name
        xtype = ncid_inp.variables[ivar].dtype
        dimids = ncid_inp.variables[ivar].dimensions
        natts = len(ncid_inp.variables[ivar].ncattrs())
        # if len(dimids)>0:
        #     if lonlat_found==1:
        #         if all(x in dimids for x in ['lsmlat', 'lsmlon']):
        #             # dimids_tup2lst = list(dimids)
        #             dimids_new=[]
        #             for x in dimids:
        #                 if x in ['lsmlat','lsmlon']:
        #                     if x == 'lsmlat':
        #                         dimids_new.append('gridcell')
        #                 else:
        #                     dimids_new.append(x)
        #             dimids = tuple(dimids_new)
        # print(dimids)
        varnames.append(varname)
        # print(varname)
        varids[varid] = ncid_out.createVariable(varname,xtype,dimids)
        for iatt in ncid_inp.variables[ivar].ncattrs():
            attname = iatt
            attvalue = getattr(ncid_inp.variables[ivar], iatt)
            # print(attname, attvalue)
            ncid_out.variables[ivar].setncattr(attname, attvalue)    
    # print(ncid_out.variables)

    '''Set the global atrributes for output netcdf'''
    setattr(ncid_out, 'Created_by', os.getlogin())
    setattr(ncid_out, 'Created_on', time.strftime("%Y-%m-%d %H:%M:%S"))



    '''Copy variables'''
    for ivar in ncid_inp.variables:
        print(ivar)
        data = ncid_inp.variables[ivar][:]
        varname = ncid_inp.variables[ivar].name
        vartype = ncid_inp.variables[ivar].dtype
        vardimids = ncid_inp.variables[ivar].dimensions
        varnatts = len(ncid_inp.variables[ivar].ncattrs())


        if len(vardimids)==0:
            print('vardimids length 0',varname)
            ncid_out[ivar][:] = data
        elif len(vardimids)==1:
            print('vardimids length 1',varname)
            if 'gridcell' in ncid_inp.variables[ivar].dimensions:
                ncid_out[ivar][:] = ncid_inp.variables[ivar][grids_in_poly] 
            else:
                ncid_out[ivar][:] = ncid_inp.variables[ivar][:] 
        elif len(vardimids)==2:
            print('vardimids length 2',varname)    
            ncid_out[ivar][:] = ncid_inp[ivar][:,grids_in_poly]

        elif len(vardimids)==3:
            print('vardimids length 3',varname)
            ncid_out[ivar][:] = ncid_inp[ivar][:,:,grids_in_poly]                    

    ncid_out.close()
    ncid_inp.close()


In [3]:
def CreateDomainDataset(infile,nc_out_name,grids_in_poly):

    # infile = 'C:/Users/zahu573/D/ELM/clm-netcdf/domain_1175x1_sparse_grid_231102.nc'

    # nc_out_name = infile.split('.')[0]+'_withinWS.nc'
    print('domain:  ', nc_out_name)

    ncid_inp = nc.Dataset(infile)
    ncid_out = nc.Dataset(nc_out_name, 'w', format='NETCDF4')   



    ndims=len(ncid_inp.dimensions)
    nvars=len(ncid_inp.variables)


    '''Define dimensions'''
    for ii in ncid_inp.dimensions:
        dimname = ncid_inp.dimensions[ii].name
        ndim = len(ncid_inp.dimensions[ii])
        print(dimname,ndim)
        if dimname == 'nv':
            ndim = len(ncid_inp.dimensions[ii])
            print(ndim)
            nv = ncid_out.createDimension(dimname, ndim)
        elif ii == 'ni':
            ndim = len(lon_elm_in)  
            print(ndim)
            ni = ncid_out.createDimension(dimname, ndim)
        elif ii == 'nj':
            ndim = len(ncid_inp.dimensions[ii])
            print(ndim)
            nj = ncid_out.createDimension(dimname, ndim)

    '''Define variables'''
    varnames = []
    varids = [0]*nvars
    for (varid,ivar) in zip(range(len(varids)),ncid_inp.variables):
        varname = ncid_inp.variables[ivar].name
        xtype = ncid_inp.variables[ivar].dtype
        dimids = ncid_inp.variables[ivar].dimensions
        natts = len(ncid_inp.variables[ivar].ncattrs())
        varnames.append(varname)
        # print(varname)
        varids[varid] = ncid_out.createVariable(varname,xtype,dimids)
        for iatt in ncid_inp.variables[ivar].ncattrs():
            attname = iatt
            attvalue = getattr(ncid_inp.variables[ivar], iatt)
            print(attname, attvalue)
            ncid_out.variables[ivar].setncattr(attname, attvalue)    
    print(ncid_out.variables)

    '''Set the global atrributes for output netcdf'''
    setattr(ncid_out, 'Created_by', os.getlogin())
    setattr(ncid_out, 'Created_on', time.strftime("%Y-%m-%d %H:%M:%S")) 

    '''Copy variables'''
    for ivar in ncid_inp.variables:
        varname = ncid_inp.variables[ivar].name
        xtype = ncid_inp.variables[ivar].dtype
        dimids = ncid_inp.variables[ivar].dimensions
        natts = len(ncid_inp.variables[ivar].ncattrs())
        if len(np.shape(ncid_inp[ivar][:,grids_in_poly])) ==2:
            print(varname, np.shape(ncid_inp[ivar][:,grids_in_poly]))
            ncid_out[ivar][:] = ncid_inp[ivar][:,grids_in_poly]
        elif  len(np.shape(ncid_inp[ivar][:,grids_in_poly])) ==3:
            print(varname, np.shape(ncid_inp[ivar][:,grids_in_poly,:]))
            ncid_out[ivar][:] = ncid_inp[ivar][:,grids_in_poly,:]


    ncid_out.close()
    ncid_inp.close()

#### Load datasets

In [23]:
#load the shapefile of watershed or burn area
WS_shp = 'C:/Users/zahu573/D/MTBS/mtbs_perimeter_data/ARW_2017.shp'
sf = shapefile.Reader(WS_shp)
shapes = sf.shapes()

#load already created unstructured surface/domain dataset
in_surffile = 'C:/Users/zahu573/D/ELM/clm-netcdf/surfdata_1175x1_sparse_grid_231102_update1k_ARW_withinWS.nc'
in_domnfile = 'C:/Users/zahu573/D/ELM/clm-netcdf/domain_1175x1_sparse_grid_231102_ARW_withinWS.nc'
surfdata_sparse = nc.Dataset( in_surffile)


#### Get lat/lons

In [24]:
# get lat,lon from surface dataset
lat = surfdata_sparse.variables['LATIXY'][:]
lon = surfdata_sparse.variables['LONGXY'][:]
print(lat,lon)

[46.82083334 46.82083334 46.82083334 46.82083334 46.82916667 46.82916667
 46.82916667 46.82916667 46.8375     46.8375     46.8375     46.8375
 46.8375     46.84583334 46.84583334 46.84583334 46.84583334 46.84583334
 46.84583334 46.84583334 46.85416667 46.85416667 46.85416667 46.85416667
 46.85416667 46.85416667 46.85416667 46.85416667 46.8625     46.8625
 46.8625     46.8625     46.8625     46.8625     46.8625     46.8625
 46.8625     46.8625     46.8625     46.87083334 46.87083334 46.87083334
 46.87083334 46.87083334 46.87083334 46.87083334 46.87083334 46.87083334
 46.87083334 46.87083334 46.87083334 46.87083334 46.87083334 46.87083334
 46.87916667 46.87916667 46.87916667 46.87916667 46.87916667 46.87916667
 46.87916667 46.87916667 46.87916667 46.87916667 46.87916667 46.87916667
 46.87916667 46.87916667 46.87916667 46.87916667 46.87916667 46.87916667
 46.87916667 46.87916667 46.87916667 46.87916667 46.8875     46.8875
 46.8875     46.8875     46.8875     46.8875     46.8875     46.887

In [25]:
# get lat,lon from shapefile

lon_shp = [[],[]]
lat_shp = [[],[]]
polygons_list = []
for shape,i in zip(shapes,range(len(shapes))):
    print(shape,i)
    print(len(shape.points))
    polygons_list.append(shape)
    for vertex in shape.points:
        print(vertex)
        #Adding 360 to lon because lon values in the shapefile ranges -180 to 180,
        #while in netcdf the values range 0 to 360
        lon_shp[i].append(vertex[0]+360)
        lat_shp[i].append(vertex[1])  

Shape #0: POLYGON 0
420
(-121.31665759137906, 46.94361448726636)
(-121.31594374678667, 46.94348693166967)
(-121.31466899571467, 46.94318756115291)
(-121.31334924427637, 46.943378499295136)
(-121.31302703363207, 46.943841964936155)
(-121.31257275188608, 46.94455887532348)
(-121.31205692491318, 46.9448419581862)
(-121.31134381237041, 46.944852488369065)
(-121.3106755122352, 46.94473882372836)
(-121.30977997994563, 46.94450720826961)
(-121.30886555735884, 46.9443984611317)
(-121.30832660157546, 46.94389296379239)
(-121.30746393507559, 46.943863442588054)
(-121.30703414011907, 46.9433808834026)
(-121.30659966687995, 46.94278721725345)
(-121.3055207804461, 46.94230225507471)
(-121.3050068429526, 46.941522256220765)
(-121.30411397535954, 46.94107356107446)
(-121.30439464263844, 46.94064899208877)
(-121.30491047051072, 46.94036594070235)
(-121.30556285171463, 46.94042882772044)
(-121.30664837695701, 46.941242829109456)
(-121.3075433516658, 46.94129706283354)
(-121.30782171487704, 46.940685770

#### Create polygons using the shape vertices and points for lat/lon of netcdf grids. check if the point is inside the polygon and store in a list

In [26]:

lat_lon_in = []
for i in range(len(shapes)):
    for lon_w,lat_w in zip(lon,lat):
        lons_lats_vect = np.column_stack((lon_shp[i], lat_shp[i]))
        polygon = Polygon(lons_lats_vect) 
        point = Point(lon_w,lat_w)
        if  polygon.contains(point)==True:
            lat_lon_in.append([lon_w,lat_w])
            # lat_lon_in.append([lon_w-360,lat_w])
lon_in = []
lat_in =[]
for k,l in zip(surfdata_sparse.variables['LONGXY'][:],surfdata_sparse.variables['LATIXY'][:]):
    # print([k,l])
    if [k,l] in lat_lon_in:
        lon_in.append(k)
        lat_in.append(l)

#### get indexes of grids inside polygon

In [27]:
grids_in_poly = []
for k,l in zip(lon_in,lat_in):
    indx = np.where((surfdata_sparse.variables['LONGXY'][:] == k) & (surfdata_sparse.variables['LATIXY'][:] == l))
    if len(indx[0])>0:
        # print(indx)
        grids_in_poly.append(indx[0][0])
lon_elm_in =surfdata_sparse.variables['LONGXY'][grids_in_poly]
lat_elm_in = surfdata_sparse.variables['LATIXY'][grids_in_poly]
print(len(lon_elm_in))

113


#### Save new surf/domain file inside shapefile boundary

In [28]:
surf_out_name = in_surffile.split('.')[0]+'_ARW_burn.nc'
domn_out_name = in_domnfile.split('.')[0]+'_ARW_burn.nc'
surfdata = CreateSurfaceDataset(surfdata_sparse,surf_out_name,grids_in_poly)
domaindata = CreateDomainDataset(in_domnfile,domn_out_name,grids_in_poly)

surface dataset:   C:/Users/zahu573/D/ELM/clm-netcdf/surfdata_1175x1_sparse_grid_231102_update1k_ARW_withinWS_ARW_burn.nc
mxsoil_color
vardimids length 0 mxsoil_color
SOIL_COLOR
vardimids length 1 SOIL_COLOR
PCT_SAND
vardimids length 2 PCT_SAND
PCT_CLAY
vardimids length 2 PCT_CLAY
ORGANIC
vardimids length 2 ORGANIC
FMAX
vardimids length 1 FMAX
natpft
vardimids length 1 natpft
LANDFRAC_PFT
vardimids length 1 LANDFRAC_PFT
PFTDATA_MASK
vardimids length 1 PFTDATA_MASK
PCT_NATVEG
vardimids length 1 PCT_NATVEG
PCT_CROP
vardimids length 1 PCT_CROP
PCT_NAT_PFT
vardimids length 2 PCT_NAT_PFT
MONTHLY_LAI
vardimids length 3 MONTHLY_LAI
MONTHLY_SAI
vardimids length 3 MONTHLY_SAI
MONTHLY_HEIGHT_TOP
vardimids length 3 MONTHLY_HEIGHT_TOP
MONTHLY_HEIGHT_BOT
vardimids length 3 MONTHLY_HEIGHT_BOT
time
vardimids length 1 time
AREA
vardimids length 1 AREA
LONGXY
vardimids length 1 LONGXY
LATIXY
vardimids length 1 LATIXY
EF1_BTR
vardimids length 1 EF1_BTR
EF1_FET
vardimids length 1 EF1_FET
EF1_FDT
vardimid

In [32]:
#checking values
dom=xr.open_dataset('C:/Users/zahu573/D/ELM/clm-netcdf/domain_1175x1_sparse_grid_231102_withinWS.nc')

In [38]:
dom['yc'].values

array([[46.82083334, 46.82083334, 46.82083334, 46.82083334, 46.82916667,
        46.82916667, 46.82916667, 46.82916667, 46.8375    , 46.8375    ,
        46.8375    , 46.8375    , 46.8375    , 46.84583334, 46.84583334,
        46.84583334, 46.84583334, 46.84583334, 46.84583334, 46.84583334,
        46.85416667, 46.85416667, 46.85416667, 46.85416667, 46.85416667,
        46.85416667, 46.85416667, 46.85416667, 46.8625    , 46.8625    ,
        46.8625    , 46.8625    , 46.8625    , 46.8625    , 46.8625    ,
        46.8625    , 46.8625    , 46.8625    , 46.8625    , 46.87083334,
        46.87083334, 46.87083334, 46.87083334, 46.87083334, 46.87083334,
        46.87083334, 46.87083334, 46.87083334, 46.87083334, 46.87083334,
        46.87083334, 46.87083334, 46.87083334, 46.87083334, 46.87916667,
        46.87916667, 46.87916667, 46.87916667, 46.87916667, 46.87916667,
        46.87916667, 46.87916667, 46.87916667, 46.87916667, 46.87916667,
        46.87916667, 46.87916667, 46.87916667, 46.8